In [3]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import re
import contractions
from spellchecker import SpellChecker

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sata\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sata\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
df = pd.read_csv('Mental-Health-Twitterr.csv')
df

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets
0,0,6.380000e+17,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1.013187e+09,84,211,251,837,0
1,1,6.380000e+17,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1.013187e+09,84,211,251,837,1
2,2,6.380000e+17,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1.013187e+09,84,211,251,837,0
3,3,6.380000e+17,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1.013187e+09,84,211,251,837,2
4,4,6.380000e+17,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1.013187e+09,84,211,251,837,1
...,...,...,...,...,...,...,...,...,...,...
19995,19995,8.190000e+17,Thu Jan 12 00:14:56 +0000 2017,A day without sunshine is like night.,1.169876e+09,442,230,7,1063601,0
19996,19996,8.190000e+17,Thu Jan 12 00:06:18 +0000 2017,"Boren's Laws: (1) When in charge, ponder. (2) ...",1.169876e+09,442,230,7,1063601,0
19997,19997,8.190000e+17,Thu Jan 12 00:05:42 +0000 2017,The flow chart is a most thoroughly oversold p...,1.169876e+09,442,230,7,1063601,0
19998,19998,8.190000e+17,Thu Jan 12 00:05:22 +0000 2017,"Ships are safe in harbor, but they were never ...",1.169876e+09,442,230,7,1063601,0


In [5]:
df=df["post_text"]
df.head()


0    It's just over 2 years since I was diagnosed w...
1    It's Sunday, I need a break, so I'm planning t...
2    Awake but tired. I need to sleep but my brain ...
3    RT @SewHQ: #Retro bears make perfect gifts and...
4    It’s hard to say whether packing lists are mak...
Name: post_text, dtype: object

In [6]:
df.isnull().sum()

0

# Text preprocessing


In [7]:
# Convert tweets characters to lowercase
df = df.str.lower()
#expands shortened words
df = df.apply(lambda x: contractions.fix(x))

# Remove numbers from tweets as they do not indicate anything
df = df.str.replace('\d+', '', regex=True)

# Remove punctuation from tweets
df = df.str.replace("[^\w\s]", "", regex=True)

# Remove URLs from tweets
df = df.str.replace(r"http\S+|www\S+|https\S+", "", regex=True)
    
df.head()

0    it is just over  years since i was diagnosed w...
1    it is sunday i need a break so i am planning t...
2    awake but tired i need to sleep but my brain h...
3    rt sewhq retro bears make perfect gifts and ar...
4    it is hard to say whether packing lists are ma...
Name: post_text, dtype: object

In [8]:
def remove_stopwords_and_lemmatize(tweets):
    filtered_tokens = []
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(tweets)
    for tweet in tokens:
        if tweet not in stop_words:
            lemmatized_word = lemmatizer.lemmatize(tweet)
            filtered_tokens.append(lemmatized_word)
#     filtered_text = ' '.join(filtered_tokens)
#     Return the filtered text
    return filtered_tokens
        
df=df.apply(remove_stopwords_and_lemmatize)    
    
df.head()

0    [year, since, diagnosed, anxiety, depression, ...
1    [sunday, need, break, planning, spend, little,...
2             [awake, tired, need, sleep, brain, idea]
3    [rt, sewhq, retro, bear, make, perfect, gift, ...
4    [hard, say, whether, packing, list, making, li...
Name: post_text, dtype: object

In [ ]:
def correct_spelling(word_list):
    spell = SpellChecker()
    corrected_list = []
    for word in word_list:
        corrected_word = spell.correction(word)
        corrected_list.append(corrected_word)
    return corrected_list

df=df.apply(correct_spelling)    
    
df.head()

# Analysis


In [ ]:
#classify the tweets based on it's polarity to depressed'1' or not '0' 
def classify_sentiment(word_list):
    polarity = 0
    text = ' '.join(str(word) for word in word_list)  # Join the list of words into a single string
    blob = TextBlob(text)
    polarity += blob.sentiment.polarity
    if polarity > .1:
        return 0
    else:
        return 1
# Apply sentiment classification to the 'text' column in the DataFrame
df['sentiment']=df['post_text'].apply(classify_sentiment)
labels=df['sentiment'].value_counts()
labels



In [ ]:

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)

# Assign the categorical labels directly
labels.index = ['Depressed', ' Not Depressed']

# Create the bar plot
labels.plot(kind="bar", rot=0, color=["plum", "cyan"])

# Set the y-axis label
plt.ylabel("Tweets")

# Display the plot
plt.show()


# splitting data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['post_text'], df['sentiment'], test_size=0.25, random_state=42)
X_train = [' '.join(filter(None, doc)) for doc in X_train]
X_test = [' '.join(filter(None, doc)) for doc in X_test]

type(X_train)

 # Convert the text data into numerical data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


# Train model (naive bayes)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)
sample_texts = [' not depressed', 'depressed',]
# Make predictions on the test data
y_pred = nb.predict(X_test)
# Create a list to store qualitative predictions
qualitative_predictions = []

# Map predicted labels to qualitative interpretations
for pred in y_pred:
    if pred == 0:
        qualitative_predictions.append('normal')

    else :
        qualitative_predictions.append('depressed')


# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)
#print results of random 10 test data
print("quantative_predictions :",y_pred[10:20])
print("qualitative_predictions :",qualitative_predictions[10:20])

# Plotting the quantitative results
normal_count = np.count_nonzero(y_pred == 0)
depressed_count = np.count_nonzero(y_pred == 1)
labels = ['Normal', 'Depressed']

counts = [normal_count, depressed_count]

plt.bar(labels, counts)
plt.xlabel('Prediction')
plt.ylabel('Count')
plt.title('Prediction Results')

plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, nb.predict(X_test))
cm = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nb.classes_)
cm.plot()

# trying random sentences 

In [ ]:

y_pred = nb.predict(vectorizer.transform([' love  to hate myself']))
print("depressed",y_pred)
y_pred = nb.predict(vectorizer.transform([' love  myself']))
print("Normal",y_pred)


# random forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
accuracy = rf.score(X_test, y_test)
print('Accuracy:', accuracy)
